In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [2]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [3]:
connection = create_server_connection("localhost", "root", "python456")

MySQL Database connection successful


In [4]:
#Criando um banco de dados
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")

create_database_query = "CREATE DATABASE teste"
create_database(connection, create_database_query)

Error: '1007 (HY000): Can't create database 'teste'; database exists'


In [4]:
#Função Conectando ao Banco de dados
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name, 
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

#Função para execução de consultas
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [41]:
#Criando tabelas
create_projeto_table = """
CREATE TABLE projeto (
  projeto_id INT PRIMARY KEY,
  projeto_nome VARCHAR(40) NOT NULL,
  regiao VARCHAR(40) NOT NULL,
  inicio DATE,
  fim DATE
);
 """

connection = create_db_connection("localhost", "root", "python456", "teste") # Connect to the Database
execute_query(connection, create_projeto_table) # Execute our defined query

MySQL Database connection successful
Query successful


In [42]:
create_rio_table = """
CREATE TABLE rio (
  rio_id INT PRIMARY KEY,
  rio_name VARCHAR(40) NOT NULL,
  foz_lat_long VARCHAR(20) NOT NULL,
  projeto INT
);
 """

create_coleta_table = """
CREATE TABLE coleta (
  coleta_id INT PRIMARY KEY,
  codigo VARCHAR(40) NOT NULL,
  lat_long VARCHAR(40) NOT NULL,
  data DATE,
  rio INT
);
"""

create_metadados_table = """
CREATE TABLE metadados (
  metadados_id INT PRIMARY KEY,
  metadados_nome VARCHAR(20),
  metadados_valor FLOAT(7,4),
  coleta INT
);
"""

connection = create_db_connection("localhost", "root", "python456", "teste")
execute_query(connection, create_rio_table)
execute_query(connection, create_coleta_table)
#execute_query(connection, create_dados_table)
execute_query(connection, create_metadados_table)


MySQL Database connection successful
Query successful
Query successful
Query successful


In [12]:
#Exemplo, depois elaborar
create_metagenoma_table = """
CREATE TABLE metagenoma (
  metagenoma_id INT PRIMARY KEY,
  grupo VARCHAR(20),
  tipo VARCHAR(70),
  qtd_reads INT,
  coleta INT
);
"""
connection = create_db_connection("localhost", "root", "python456", "teste")
execute_query(connection, create_metagenoma_table)

MySQL Database connection successful
Query successful


In [43]:
alter_rio = """
ALTER TABLE rio
ADD FOREIGN KEY(projeto)
REFERENCES projeto(projeto_id)
ON DELETE SET NULL;
"""
connection = create_db_connection("localhost", "root", "python456","teste")
#execute_query(connection, alter_projeto)
execute_query(connection, alter_rio)

MySQL Database connection successful
Query successful


In [44]:
#Arranjo para realizar a relação muito para muitos (N:M)

#alter_projeto = """
#ALTER TABLE projeto
#ADD FOREIGN KEY(projeto_id)
#REFERENCES rio(rio_id)
#ON DELETE SET NULL;
#"""

alter_coleta = """
ALTER TABLE coleta
ADD FOREIGN KEY(rio)
REFERENCES rio(rio_id)
ON DELETE SET NULL;
"""
alter_metadados = """
ALTER TABLE metadados
ADD FOREIGN KEY(coleta)
REFERENCES coleta(coleta_id)
ON DELETE SET NULL;
"""


#DEPOIS SE PRECISAR CRIAR RELÇÃO DE PROJETO COM RIOS (n:m)
#create_participadoprojeto_table = """
#CREATE TABLE takes_course (
 # participant_id INT,
  #course_id INT,
  #PRIMARY KEY(participant_id, course_id),
  #FOREIGN KEY(participant_id) REFERENCES participant(participant_id) ON DELETE CASCADE, -- it makes no sense to keep this rtelation when a participant or course is no longer in the system, hence why CASCADE this time
  #FOREIGN KEY(course_id) REFERENCES course(course_id) ON DELETE CASCADE
#);
#"""

connection = create_db_connection("localhost", "root", "python456","teste")
#execute_query(connection, alter_projeto)
#execute_query(connection, alter_rio)
execute_query(connection, alter_coleta)
execute_query(connection, alter_metadados)
#execute_query(connection, alter_metagenoma)


MySQL Database connection successful
Query successful
Query successful


In [13]:
alter_metagenoma = """
ALTER TABLE metagenoma
ADD FOREIGN KEY(coleta)
REFERENCES coleta(coleta_id)
ON DELETE SET NULL;
"""
connection = create_db_connection("localhost", "root", "python456","teste")
execute_query(connection, alter_metagenoma)


MySQL Database connection successful
Query successful


In [47]:
pop_projeto = """
INSERT INTO projeto VALUES
(1,  'BIG', 'Rios da Baia de Ilha Grande', '2022-01-01', '2022-12-01'),
(2,  'Guandu', 'Rio Guandu', '2019-01-01', '2020-12-01'),
(3,  'Amazonia', 'Rios da Amazonia', '2015-01-01', '2016-12-01');
"""

connection = create_db_connection("localhost", "root",  "python456","teste")
execute_query(connection, pop_projeto)

MySQL Database connection successful
Query successful


In [ ]:
#INSERÇÃO ANTIGA
pop_rio = """
INSERT INTO rio VALUES
(101, 'Guandú',  'Fictício...', 2);
"""
#(103, 'Mambucaba', 'Fictício...', 1),
#(102, 'Frade', 'Fictício...', 1),
#(104, 'Tapajós',  'Fictício...', 3),
connection = create_db_connection("localhost", "root",  "python456","teste")
execute_query(connection, pop_rio)

In [13]:

#PERCORRENDO LINHA A LINHA DA TABELA E INSERINDO NO BANCO DE DADOS
import pandas as pd

dfr = pd.read_excel('metadado.xlsx', sheet_name='rio')

for i, ide in enumerate(dfr['rio_id']):
    nome = dfr.loc[i,'rio_nome']
    geo = dfr.loc[i,'foz_lat_long']
    proj = dfr.loc[i,'projeto']
    
    dados = '{0}, \'{1}\', \'{2}\', {3})'.format(ide,nome,geo,proj)
    script = '''INSERT INTO rio VALUES ('''

    pop_rio = script + dados
    print(pop_rio)
    #cursor_dw.execute(query)
    #cursor_dw.commit()
    connection = create_db_connection("localhost", "root",  "python456","teste")
    execute_query(connection, pop_rio)

INSERT INTO rio VALUES (102, 'Mambucaba', 'teste', 1)
MySQL Database connection successful
Query successful
INSERT INTO rio VALUES (103, 'Frade', 'teste', 1)
MySQL Database connection successful
Query successful
INSERT INTO rio VALUES (104, 'Parati-mirim', 'teste', 1)
MySQL Database connection successful
Query successful
INSERT INTO rio VALUES (105, 'Centro ', 'teste', 1)
MySQL Database connection successful
Query successful
INSERT INTO rio VALUES (106, 'Jacareí', 'teste', 1)
MySQL Database connection successful
Query successful
INSERT INTO rio VALUES (107, 'Bracui', 'teste', 1)
MySQL Database connection successful
Query successful
INSERT INTO rio VALUES (108, 'Taquari', 'teste', 1)
MySQL Database connection successful
Query successful
INSERT INTO rio VALUES (109, 'Jacuecanga', 'teste', 1)
MySQL Database connection successful
Query successful
INSERT INTO rio VALUES (110, 'Japuiba', 'teste', 1)
MySQL Database connection successful
Query successful
INSERT INTO rio VALUES (111, 'São Roque

In [8]:
pop_coleta = """
INSERT INTO coleta VALUES
(1001, 'M11',  'Fictício...', '2019-01-01', 101),
(1002, 'M12',  'Fictício...', '2019-01-01', 101);
"""
#(103, 'Mambucaba', 'Fictício...', 1),
#(102, 'Frade', 'Fictício...', 1),
#(104, 'Tapajós',  'Fictício...', 3),
connection = create_db_connection("localhost", "root",  "python456","teste")
execute_query(connection, pop_coleta)

MySQL Database connection successful
Query successful


In [8]:

#PERCORRENDO LINHA A LINHA DA TABELA E INSERINDO NO BANCO DE DADOS
import pandas as pd

dfc = pd.read_excel('metadado.xlsx', sheet_name='coleta')

for i, ide in enumerate(dfc['coleta_id']):
    nome = dfc.loc[i,'codigo']
    geo = dfc.loc[i,'lat_long']
    data = dfc.loc[i,'data']
    proj = dfc.loc[i,'rio']
    
    dados = '{0}, \'{1}\', \'{2}\',\'{3}\', {4})'.format(ide,nome,geo,data,proj)
    script = '''INSERT INTO coleta VALUES ('''

    pop_coleta = script + dados
    print(pop_coleta)
    #cursor_dw.execute(query)
    #cursor_dw.commit()
    connection = create_db_connection("localhost", "root",  "python456","teste")
    execute_query(connection, pop_coleta)

INSERT INTO coleta VALUES (1003, 'PAR A', 'teste','2022-02-03', 104)
MySQL Database connection successful
Error: '1062 (23000): Duplicate entry '1003' for key 'PRIMARY''
INSERT INTO coleta VALUES (1004, 'CJA A', 'teste','2022-02-03', 106)
MySQL Database connection successful
Error: '1062 (23000): Duplicate entry '1004' for key 'PRIMARY''
INSERT INTO coleta VALUES (1005, 'FOZ A', 'teste','2022-02-03', 111)
MySQL Database connection successful
Error: '1062 (23000): Duplicate entry '1005' for key 'PRIMARY''
INSERT INTO coleta VALUES (1006, 'MAM A', 'teste','2022-02-03', 102)
MySQL Database connection successful
Error: '1062 (23000): Duplicate entry '1006' for key 'PRIMARY''
INSERT INTO coleta VALUES (1007, 'FRA A', 'teste','2022-02-03', 103)
MySQL Database connection successful
Error: '1062 (23000): Duplicate entry '1007' for key 'PRIMARY''
INSERT INTO coleta VALUES (1008, 'TAQ A', 'teste','2022-02-03', 108)
MySQL Database connection successful
Error: '1062 (23000): Duplicate entry '1008'

In [10]:
#Exercicio de como inserir excel dentro do bando de dados
import pandas as pd

df = pd.read_excel('metadado.xlsx')

#APAGAR REGISTROS NA TABELA
#cursor_dw.execute('TRUNCATE TABLE metadados')
#cursor_dw.commit()

#PERCORRENDO LINHA A LINHA DA TABELA E INSERINDO NO BANCO DE DADOS
for i, ide in enumerate(df['metadado_id']):
    nome = df.loc[i,'metadado_nome']
    valor = df.loc[i,'metadado_valor']
    coleta = df.loc[i,'coleta']
    
    dados = '{0}, \'{1}\', {2}, {3})'.format(ide,nome,valor,coleta)
    script = '''INSERT INTO metadados VALUES ('''

    pop_metadado = script + dados
    print(pop_metadado)
    #cursor_dw.execute(query)
    #cursor_dw.commit()
    connection = create_db_connection("localhost", "root",  "python456","teste")
    execute_query(connection, pop_metadado)

INSERT INTO metadados VALUES (10001, 'Ca (µg/mL)', 5.94367, 1001)
MySQL Database connection successful
Query successful
INSERT INTO metadados VALUES (10002, 'K (µg/mL)', 4.46617, 1001)
MySQL Database connection successful
Query successful
INSERT INTO metadados VALUES (10003, 'Mg (µg/mL)', 1.75774, 1001)
MySQL Database connection successful
Query successful
INSERT INTO metadados VALUES (10004, 'Na (µg/mL)', 9.50246, 1001)
MySQL Database connection successful
Query successful
INSERT INTO metadados VALUES (10005, 'P (µg/mL)', 0.141515, 1001)
MySQL Database connection successful
Query successful
INSERT INTO metadados VALUES (10006, 'S (µg/mL)', 4.73926, 1001)
MySQL Database connection successful
Query successful
INSERT INTO metadados VALUES (10007, 'COD (mg/L)', 4.18, 1001)
MySQL Database connection successful
Query successful
INSERT INTO metadados VALUES (10008, 'TN (mg/L)', 1.2770000000000001, 1001)
MySQL Database connection successful
Query successful
INSERT INTO metadados VALUES (10009

In [14]:
#Inserindo metagenomas
import pandas as pd

dfm = pd.read_excel('metadado.xlsx', sheet_name='metagenoma')

for i, ide in enumerate(dfm['metagenoma_id']):
    grupo = dfm.loc[i,'grupo']
    tipo = dfm.loc[i,'tipo']
    read = dfm.loc[i,'qtde_reads']
    coleta = dfm.loc[i,'coleta']
    
    dados = '{0}, \'{1}\', \'{2}\', {3}, {4})'.format(ide,grupo,tipo,read,coleta)
    script = '''INSERT INTO metagenoma VALUES ('''

    pop_metagenoma = script + dados
    print(pop_metagenoma)
    #cursor_dw.execute(query)
    #cursor_dw.commit()
    connection = create_db_connection("localhost", "root",  "python456","teste")
    execute_query(connection, pop_metagenoma)

INSERT INTO metagenoma VALUES (100000, 'genero_bacteria', 'Chitinophaga', 913, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100001, 'genero_bacteria', 'Ferruginibacter', 292, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100002, 'genero_bacteria', 'Flavihumibacter', 266, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100003, 'genero_bacteria', 'Flavipsychrobacter', 4086, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100004, 'genero_bacteria', 'Sediminibacterium', 6074, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100005, 'genero_bacteria', 'Taibaiella', 660, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100006, 'genero_bacteria', 'Aquirufa', 19246, 1030)
MySQL Database connection successful
Query successful
INSERT INTO 

MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100063, 'genero_bacteria', 'Rhizobacter', 777, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100064, 'genero_bacteria', 'Rubrivivax', 819, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100065, 'genero_bacteria', 'Sphaerotilus', 578, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100066, 'genero_bacteria', 'Tepidimonas', 330, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100067, 'genero_bacteria', 'Thiomonas', 274, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100068, 'genero_bacteria', 'Xylophilus', 331, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100069, 'genero_bacteria', 'Acidovorax', 6223, 1030)
MySQL Database connection successful


MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100130, 'genero_bacteria', 'Bdellovibrio', 392, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100131, 'genero_bacteria', 'Oleiharenicola', 239, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100132, 'genero_bacteria', 'Opitutus', 509, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100133, 'genero_bacteria', 'Pedosphaera', 351, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100134, 'genero_bacteria', 'Prosthecobacter', 218, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100135, 'genero_bacteria', 'Acidimicrobium', 246, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100136, 'genero_bacteria', 'Rhabdothermincola', 280, 1030)
MySQL Database conne

MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100196, 'arg', 'Polymyxin resistance', 35, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100197, 'arg', 'Polymyxin resistance, lipid A modifications with 4-amino-4-deoxy-L-arabinose', 234, 1030)
MySQL Database connection successful
Error: '1406 (22001): Data too long for column 'tipo' at row 1'
INSERT INTO metagenoma VALUES (100198, 'arg', 'Polymyxin resistance, lipid A modifications with phosphoethanolamine', 223, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100199, 'arg', 'Resistance to Capreomycin and Viomycin', 9, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100200, 'arg', 'Resistance to chromium compounds', 667, 1030)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100201, 'arg', 'Resistance to Daptomycin', 998, 1030)
MySQL Da

MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100262, 'genero_bacteria', 'Rhodopseudomonas', 65, 1040)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100263, 'genero_bacteria', 'Devosia', 104, 1040)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100264, 'genero_bacteria', 'Hyphomicrobium', 316, 1040)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100265, 'genero_bacteria', 'Rhodoplanes', 243, 1040)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100266, 'genero_bacteria', 'Methylobacterium', 304, 1040)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100267, 'genero_bacteria', 'Microvirga', 131, 1040)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100268, 'genero_bacteria', 'Mesorhizobium', 332, 1040)
MySQL Database connection

...

Query successful
INSERT INTO metagenoma VALUES (100451, 'genero_bacteria', 'Lacticaseibacillus', 93, 1040)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100452, 'genero_bacteria', 'Lactiplantibacillus', 79, 1040)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100453, 'genero_bacteria', 'Streptococcus', 507, 1040)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100454, 'genero_bacteria', 'Clostridium', 391, 1040)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100455, 'genero_bacteria', 'Faecalibacterium', 73, 1040)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100456, 'genero_bacteria', 'Clostridioides', 183, 1040)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100457, 'genero_bacteria', 'Candidatus Nitrosotalea', 119, 1040)
MySQL Database connection successful


MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100518, 'genero_bacteria', 'Chitinophaga', 317, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100519, 'genero_bacteria', 'Deminuibacter', 35, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100520, 'genero_bacteria', 'Ferruginibacter', 123, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100521, 'genero_bacteria', 'Filimonas', 101, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100522, 'genero_bacteria', 'Flavihumibacter', 207, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100523, 'genero_bacteria', 'Flavipsychrobacter', 134, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100524, 'genero_bacteria', 'Flavisolibacter', 110, 1020)
MySQL Database 

Query successful
INSERT INTO metagenoma VALUES (100579, 'genero_bacteria', 'Roseomonas', 98, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100580, 'genero_bacteria', 'Azospirillum', 107, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100581, 'genero_bacteria', 'Magnetospirillum', 55, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100582, 'genero_bacteria', 'Candidatus Megaira', 39, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100583, 'genero_bacteria', 'Erythrobacter', 35, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100584, 'genero_bacteria', 'Aestuariisphingobium', 772, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100585, 'genero_bacteria', 'Aquisediminimonas', 57, 1020)
MySQL Database connection successful
Query succ

MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100641, 'genero_bacteria', 'Noviherbaspirillum', 69, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100642, 'genero_bacteria', 'Undibacterium', 700, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100643, 'genero_bacteria', 'Aquitalea', 80, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100644, 'genero_bacteria', 'Chromobacterium', 68, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100645, 'genero_bacteria', 'Candidatus Methylopumilus', 642, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100646, 'genero_bacteria', 'Methylobacillus', 35, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100647, 'genero_bacteria', 'Methylophilus', 38, 1020)
MySQL Da

MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100705, 'genero_bacteria', 'Arthrobacter', 54, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100706, 'genero_bacteria', 'Micromonospora', 34, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100707, 'genero_bacteria', 'Nocardioides', 75, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100708, 'genero_bacteria', 'Streptomyces', 233, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100709, 'genero_bacteria', 'Armatimonas', 34, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100710, 'genero_bacteria', 'Leptolyngbya', 35, 1020)
MySQL Database connection successful
Query successful
INSERT INTO metagenoma VALUES (100711, 'genero_bacteria', 'Pseudanabaena', 46, 1020)
MySQL Database connection succe

In [13]:
#Atualizando
update = """
UPDATE client 
SET address = '23 Fingiertweg, 14534 Berlin' 
WHERE client_id = 101;
"""

connection = create_db_connection("localhost", "root", "python456","school")
execute_query(connection, update)

MySQL Database connection successful
Query successful


In [14]:
#Deletando
delete_course = """
DELETE FROM course 
WHERE course_id = 20;
"""

connection = create_db_connection("localhost", "root", "python456","school")
execute_query(connection, delete_course)

#tem  um comando que dá pra restaurar também,depois de deletar

MySQL Database connection successful
Query successful


In [16]:
#INSERINDO DIVERSOS VALORES DE UMA LISTA COM EXECUTEMANY
def execute_list_query(connection, sql, val):
    cursor = connection.cursor()
    try:
        cursor.executemany(sql, val)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")
        
sql = '''
    INSERT INTO teacher (teacher_id, first_name, last_name, language_1, language_2, dob, tax_id, phone_no) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    '''
    
val = [
    (7, 'Hank', 'Dodson', 'ENG', None, '1991-12-23', 11111, '+491772345678'), 
    (8, 'Sue', 'Perkins', 'MAN', 'ENG', '1976-02-02', 22222, '+491443456432')
]

connection = create_db_connection("localhost", "root", "python456","school")
execute_list_query(connection, sql, val)

MySQL Database connection successful
Query successful


In [11]:
#Lendo dados
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

q1 = """
SELECT *
FROM teacher;
"""

connection = create_db_connection("localhost", "root", "python456","school")
results = read_query(connection, q1)

for result in results:
    print(result)        

MySQL Database connection successful
(1, 'James', 'Smith', 'ENG', None, datetime.date(1985, 4, 20), 12345, '+491774553676')
(2, 'Stefanie', 'Martin', 'FRA', None, datetime.date(1970, 2, 17), 23456, '+491234567890')
(3, 'Steve', 'Wang', 'MAN', 'ENG', datetime.date(1990, 11, 12), 34567, '+447840921333')
(4, 'Friederike', 'Müller-Rossi', 'DEU', 'ITA', datetime.date(1987, 7, 7), 45678, '+492345678901')
(5, 'Isobel', 'Ivanova', 'RUS', 'ENG', datetime.date(1963, 5, 30), 56789, '+491772635467')
(6, 'Niamh', 'Murphy', 'ENG', 'IRI', datetime.date(1995, 9, 8), 67890, '+491231231232')


In [17]:
#CONTEUDO EXTRA PARA VISUALIZAR EXTRAÇÃO DE COTEUDO
#Inicializa uma lista vazia 
from_db = []

# Percorrer os resultados e inseri-los à lista

# Retorna uma lista de tuplas
for result in results:
  result = result
  from_db.append(result)
    
# Retorna uma lista de listas
from_db = []

for result in results:
  result = list(result)
  from_db.append(result)



In [ ]:
#ATENÇÃO: A LEITURA E TRATAMENTO DE DADOS ESTÁ NO NOTEBOOK!!!